In this notebook, we'll make use of the _folium_ library in combination with the _pandas_ library to build a choropleth showing population by zip code for each zip code in Davidson County.

In [ ]:
import folium
import pandas as pd

Creating an interactive map is as simple as creating as calling `.Map()` from the folium library.

In [ ]:
folium.Map()

Of course, if we are planning to show something about Davidson County, we don't want our map to be zoomed all the way out.

When creating out map, we can set the `location` and `zoom_start` attribute when creating our map. It also makes sense to save the map to a variable so that we can continue to modify it.

In [ ]:
nashville = [36.1636,-86.7823]       # Latitude/Longitude of the approximately the middle of Nashville

m = folium.Map(location=nashville,   # Create the map
               zoom_start=10)

m                             # display the map

To create our choropleth, we need two things:
1. A DataFrame containing the data we want to be displayed on the choropleth (this data is contained in the file Davidson_Population.csv).
2. A geojson file containing polygons of the regions we want to show on our map (this is contained in the file Zip Codes.geojson).

First, let's read in the unemployment data and take a look.

In [ ]:
population = pd.read_csv('../data/Davidson_Population.csv')
population.head(2)

To build our choropleth, we are going to use the Choropleth class from `folium`. Let's take a look at the docstring.

In [ ]:
folium.Choropleth?

There are quite a few arguments that we could pass in.

Let's start simple and just get a basic map working. To do this, we need to point the Choropleth to our geojson.

Note also that we need to add the Choropleth to the empty map that we created.

In [ ]:
nashville = [36.1636,-86.7823]
m = folium.Map(location=nashville, zoom_start=10)      # Start with an empty map

zip_geo = '../data/Zip Codes.geojson'                  # Filepath to our geojson

choro = folium.Choropleth(
    geo_data=zip_geo
).add_to(m)                                                 # Tell folium to add this to our existing map

m

We now have a basic choropleth, but it is only showing the counties as it not connected to our data at all.

To connect the Choropleth to our unemployment data, there are 3 additional arguments we need to use:

* **data:** Tell folium where to find the data we want plotted on our map
* **key_on:** Where in the geojson file to find the matching key to join together our DataFrame and geojson.
* **columns:** Tell folium which columns to look in for 1) identifying the zip code (this should match the key_on value from above) and 2) coloring the polygon

The `data` argument is the easiest one - the data is contained in our `population` DataFrame.

To determine the `key_on` value, we need to take a peek at the geojson file. The beginning of this file looks like this:

```
{
  "type": "FeatureCollection",
  "features": [
    {"type":"Feature","properties":{"zip":"37115","objectid":"1","po_name":"MADISON","shape_stlength":"178783.02488886821","shape_starea":"596553400.57885742"},"geometry":{"type":"MultiPolygon","coordinates":[[[[-86.68724897141,36.318212121092],
```

We'll be matching each row of our dataframe with the matching polygon through the zip code. We need to give folium directions to access the zip code in the geojson. This will look like `feature.properties.zip`.

Finally, we need to specify the columns. The column matching our `key_on` value is "Zip" and the values we want to plot are in the "Population_2019" column. Thus, the argument should be the list `["Zip", "Population_2019"]`.

In [ ]:
nashville = [36.1636,-86.7823]
m = folium.Map(location=nashville, zoom_start=10)                  

zip_geo = '../data/Zip Codes.geojson' 
choro = folium.Choropleth(
    geo_data=zip_geo,
    data=population,
    key_on="feature.properties.zip",
    columns=["Zip", "Population_2019"]
).add_to(m)

m

What happened?

The zip codes are now plotted, and the legend is correct, but the values didn't translate correctly to the polygons. It seems that folium was not able to correctly match the polygons to the values in the DataFrame.

The cause of this is that in the dataframe, the zip code is treated as an integer but in the geojson file, it is treated as a string. We can fix this by changing the type to a string in our dataframe.

In [ ]:
population['Zip'] = population['Zip'].astype('str')

In [ ]:
nashville = [36.1636,-86.7823]
m = folium.Map(location=nashville, zoom_start=10)                  

zip_geo = '../data/Zip Codes.geojson' 
choro = folium.Choropleth(
    geo_data=zip_geo,
    data=population,
    key_on="feature.properties.zip",
    columns=["Zip", "Population_2019"]
).add_to(m)

m

There are a couple of other improvements we could make. 

First, the legend should indicate what our choropleth is showing. For this, we can use the `legend_name` argument.

Second, it would be nice if we could identify which zip code is which. To do this, we can use the `add_child` method and point this method to the "zip" field in our geojson file.

In [ ]:
nashville = [36.1636,-86.7823]
m = folium.Map(location=nashville, zoom_start=10)                  

zip_geo = '../data/Zip Codes.geojson' 
choro = folium.Choropleth(
    geo_data=zip_geo,
    data=population,
    key_on="feature.properties.zip",
    columns=["Zip", "Population_2019"],
    legend_name = 'Population in 2019'
).add_to(m)

choro.geojson.add_child(
   folium.features.GeoJsonTooltip(['zip'])
)

m